# Bayesian Personalized Ranking (BPR)

Bayesian Personalized Ranking (BPR)は、機械学習の分野で開発された、主に推薦システムに用いられるアルゴリズムです。ユーザーの好みをペアワイズのランキング問題としてモデル化し、ベイジアン統計学の枠組みを使用して、ユーザーごとにパーソナライズされたアイテムのランキングを生成します。この手法は特に、暗黙のフィードバックデータ（例えば、購入履歴や閲覧履歴）に基づいて、アイテムの好みを予測するのに有効です。
- GitHub：https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/cornac_bpr_deep_dive.ipynb

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/recommendation/collaborative_filtering(linear)/bpr.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture
!pip install cornac==1.17.0

In [32]:
import cornac
from cornac.data import Reader
from cornac.datasets import netflix
from cornac.eval_methods import RatioSplit

### データの用意

In [33]:
feedback = netflix.load_feedback(variant="small", reader=Reader(bin_threshold=1.0))


ratio_split = RatioSplit(
    data=feedback,
    test_size=0.1,
    rating_threshold=1.0,
    exclude_unknowns=True,
    verbose=True,
)

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 9988
Number of items = 4925
Number of ratings = 547022
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 8206
Number of items = 3374
Number of ratings = 60756
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 9988
Total items = 4925


### Bayesian Personalized Ranking

In [34]:
bpr = cornac.models.BPR(k=50, max_iter=200, learning_rate=0.001, lambda_reg=0.001, verbose=True)

# Use AUC and Recall@20 for evaluation
auc = cornac.metrics.AUC()
rec_20 = cornac.metrics.Recall(k=20)


### 学習と予測

In [35]:
exp =cornac.Experiment(
    eval_method=ratio_split,
    models=[bpr],
    metrics=[auc, rec_20],
    user_based=True,
).run()


[BPR] Training started!


  0%|          | 0/200 [00:00<?, ?it/s]

Optimization finished!

[BPR] Evaluation started!


Ranking:   0%|          | 0/8206 [00:00<?, ?it/s]


TEST:
...
    |    AUC | Recall@20 | Train (s) | Test (s)
--- + ------ + --------- + --------- + --------
BPR | 0.9331 |    0.2330 |  111.2309 |  62.5549

